# Importing Dependencies

In [3]:
import cv2
import mediapipe as mp   #liabrary using for posture detection
import numpy as np
mp_drawing = mp.solutions.drawing_utils  #gives us all the drawing utilities
mp_pose = mp.solutions.pose

# Making Detection

In [4]:
cap = cv2.VideoCapture(0)

## setup mediapipe instance
with mp_pose.Pose(min_detection_confidence = 0.5,min_tracking_confidence = 0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read()
        
        # recolor image to RGB
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # making detection
        results = pose.process(image)
        
        # recolor image to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        
        # extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            #get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            #calculating angle
            angle = calculate_angle(shoulder,elbow,wrist)
            
            # visualize
            cv2.putText(image,str(angle),tuple(np.multiply(elbow,[640,480]).astype(int),
                                              cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),2,cv2.LINE_AAA))
        except:
            pass
        
        # render detections
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color = (245,150,230),thickness = 2,circle_radius = 2),
                                  mp_drawing.DrawingSpec(color = (245,66,150),thickness = 2,circle_radius = 2)
                                 )
        
        cv2.imshow('Mediapipe Feed',image)
    
        if cv2.waitKey(10) & 0xFF == ord('s'):
            break
        
    cap.release()
    cv2.destroyAllWindows()